## Analiz için Sorular
* Vardiya analizi
Hangi vardiyalarda daha yüksek verimlilik elde ediliyor?
Hangi makine veya gruplar daha yüksek verimlilik gösteriyor?
Vardiyaların verimlilik sıralaması nasıl? 1>2>3 gibi mi?
* Gün analizi
Haftanın hangi günleri daha fazla verimlilik elde ediliyor?
Hafta içi mi, yoksa hafta sonu mu daha iyi verimlilik sağlıyor?
İş günleri ve hafta sonu arasındaki verimlilik farkı nedir???
* Ürün analizi
Hangi ürünler daha fazla üretim yapılıyor?
Hangi makineler hangi ürünleri üretebiliyor?
Ürünlerin desen zorluğu skorları nedir?
Ürünlerin verimlilikleri birbiriyle nasıl ilişkili?
* Operatör analizi
Operatörlerin performansı nasıl??
Hangi operatörler daha yüksek verimlilik sağlıyor?
Operatörlerin vardiya bazındaki performansı nasıl değişiyor?
* Makine analizi
* Nasıl iyileştirmeler yapılabilir?
Hangi faktörler verimliliği etkiliyor?
Hangi faktörler iyileştirilebilir ve verimliliği artırabilir?
Hangi makineler veya vardiyalar daha fazla dikkate alınmalıdır?

# Veri hazırlığı

In [ ]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [ ]:
# Verileri yükleme
df=pd.read_csv("/home/ubuntu/zendesk/merged.csv")

* Değişken ekleme / verileri hazırlama



In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

df["YarnCode"] = df["YarnCode"].astype("object")
df["Operator"] = df["Operator"].astype("object")
df["Shift"] = df["Shift"].astype("object")

# Aylık veriler
df_aylik = df.resample('M', on='Date').mean()

# Haftalık veriler
df_haftalik = df.resample('W', on='Date').mean()

 # df.set_index('Date', inplace=True)
#Yıl,ay ve haftanın günleri ve vardiya saati
df['Yıl'] = df["Date"].dt.year
df['Ay'] = df["Date"].dt.month
df['Haftanın_Günü'] = df["Date"].dt.weekday
haftanin_gunleri = {0: 'Pazartesi', 1: 'Salı', 2: 'Çarşamba', 3: 'Perşembe', 4: 'Cuma', 5: 'Cumartesi', 6: 'Pazar'}
df['Haftanın_Günü'] = df["Date"].dt.weekday.map(haftanin_gunleri)
df['Vardiya_Saati'] = df['Shift'].map({1: 'Sabah', 2: 'Akşam', 3: 'Gece'})

#Yıllık ve aylık ortalama verimlilikler dataframe şeklinde
Yıllık_veri = df.groupby("Yıl")['Efficiency'].mean().reset_index()
aylık_veri = df.groupby(["Yıl","Ay"])['Efficiency'].mean().reset_index()
veri_2022 = df[df['Yıl'] == 2022]
veri_2023 = df[df['Yıl'] == 2023]

# Haftanın günlerine göre ortalama verimlilik

In [ ]:
df_gunluk_ortalama = df.groupby([df["Date"].dt.date,"Vardiya_Saati"])[['Efficiency',"CycleTime"]].mean().reset_index()
df_gunluk_ortalama.head()

In [ ]:

Gün_bazında_verimlilikler = df.groupby("Haftanın_Günü",as_index=False)["Efficiency"].mean()

plt.figure(figsize=(10, 6))
ax = sns.barplot(data=Gün_bazında_verimlilikler, x="Haftanın_Günü", y="Efficiency")
plt.title("Haftanın Günlerine Göre Efficiency Dağılımı")
plt.xlabel("Haftanın_Günü")
plt.ylabel("Efficiency")
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')
ax.plot([-0.5, 6.5], [88.81, 89.74], color='red', linestyle='--', linewidth=1)

plt.show()

# AYLIK VE HAFTALIK VERİMLİLİK GRAFİĞİ

In [ ]:
plt.figure(figsize=(12, 6))
ax1 = plt.gca()
ax2 = ax1.twinx()

ax1.plot(df_aylik.index, df_aylik['Efficiency'], marker='o', linestyle='-', color='blue', label='Aylık')
ax2.plot(df_haftalik.index, df_haftalik['Efficiency'], marker='o', linestyle='-', color='green', label='Haftalık')

ax1.set_ylabel('Aylık Verimlilik', color='blue')
ax2.set_ylabel('Haftalık Verimlilik', color='green')

plt.title('Aylık ve Haftalık Verimlilik Grafiği')
plt.show()

# Operatör analizi

Vardiyalara göre Efficiency grafiği

In [ ]:

plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df.groupby("Vardiya_Saati")["Efficiency"].mean().reset_index(), x="Vardiya_Saati", y="Efficiency", width=0.7)
plt.title("Vardiyalara Göre Efficiency Dağılımı")
plt.xlabel("Vardiya_Saati")
plt.ylabel("Efficiency")
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')
ax.plot([-0.5, 6.5], [88.81, 89.74], color='red', linestyle='--', linewidth=1)

plt.show()

Ortalama verimliliklerin belirlenmesi ve bir verimlilik scalası belirlenmesi

In [ ]:
operatör_verimlilik_ortalamaları = df.groupby("Operator")["Efficiency"].mean().reset_index()
operatör_verimlilik_ortalamaları["Efficiency_Grup"] = pd.cut(operatör_verimlilik_ortalamaları["Efficiency"], bins=[0, 25, 50, 75, 100], labels=['1-25', '26-50', '51-75', '76-100'])

operatör_grupları = operatör_verimlilik_ortalamaları.groupby("Efficiency_Grup")["Operator"].unique()
df_operatör_grupları = operatör_grupları.reset_index(name='Operatörler')

Operatör verimlilikleri en çok verimli olana doğru sıralama


In [ ]:
operatör_verimlilik_ortalamaları.sort_values(by="Efficiency", ascending=True) 

Efficiency gruplarına göre operatörlere ulaşma

In [ ]:

df_operatör_grupları[df_operatör_grupları["Efficiency_Grup"] == '76-100']['Operatörler'].values[0]

Genel olarak Operatörlerin vardiya bazındaki performansı nasıl değişiyor? Bunun incelenmesi

In [ ]:
Vardiya_performansı = df.groupby(['Haftanın_Günü', 'Operator', 'Vardiya_Saati'])['Efficiency'].mean().reset_index()


plt.figure(figsize=(12, 6))
sns.boxplot(x='Vardiya_Saati', y='Efficiency', data=Vardiya_performansı)
plt.title('Operatörlerin Vardiya Bazında Performansı')
plt.xlabel('Vardiya')
plt.ylabel('Günlük Performans')
plt.show()

Aykırı değerleri daha iyi gözlemlemek için

In [ ]:


sns.stripplot(data=Vardiya_performansı, x="Efficiency", y="Haftanın_Günü", hue="Haftanın_Günü", legend=False)

Örneğin herhangi bir operatorun vardiya performansının incelenmesi

In [ ]:
def operator_performans_görseli(df, operator_id):
    operator = df[df["Operator"] == operator_id]
    performans = operator.groupby(["Haftanın_Günü", "Vardiya_Saati"])["Efficiency"].mean().reset_index()
    plt.figure(figsize=(12, 5))
    ax = sns.barplot(x="Vardiya_Saati", y='Efficiency', data=performans)
    plt.title(f"{operator_id} Nolu Operatörün Vardiya Bazında Performansı")
    plt.xlabel("Vardiya")
    plt.ylabel("Günlük Performans")
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

    plt.show()


operator_performans_görseli(Vardiya_performansı, 23382)

# Makinaların analizi

Genel olarak makinaların verimliliklerini gözlemleme

In [ ]:

mach_verimlilikler = df.groupby('MachCode')["Efficiency"].mean().reset_index()
mach_verimlilikler.sort_values(by="Efficiency",ascending=False)

Aylara göre bir dataframe oluşturma

In [ ]:
mach_gruplar = df.groupby(['MachCode',"Ay"])["Efficiency"].mean().reset_index()
mach_gruplar

Hangi makine kaç farklı ürün üretmiş? Bunun analizi

In [ ]:


makine_urun_sayısı = df.groupby("MachCode")["ProductCode"].nunique().reset_index()
makine_urun_sayısı = makine_urun_sayısı.sort_values(by="ProductCode", ascending=False)

plt.figure(figsize=(12, 6))
barplot= plt.bar(range(1, len(makine_urun_sayısı) + 1), makine_urun_sayısı["ProductCode"], color="skyblue")
plt.xlabel("Makine Kodu")
plt.ylabel("Farklı Ürün Sayısı")
plt.title("Her Makinenin Üretebildiği Farklı Ürün Sayısı")
plt.xticks(range(1, 25), range(1, 25))
for rect in barplot:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2, height, str(int(height)), ha="center", va="bottom")
plt.xticks(range(1, len(makine_urun_sayısı) + 1), makine_urun_sayısı["MachCode"])

plt.show()

Tüm Makinelerin aylara göre verimlilik grafiğini veren fonksiyon

In [ ]:
def makine_verimlilik(MachCode):
    plt.figure(figsize=(12, 6))
    veri_makine = mach_gruplar[mach_gruplar["MachCode"] == MachCode]
    aylar = veri_makine["Ay"].unique()
    ortalama_verimlilik = veri_makine.groupby("Ay")["Efficiency"].mean()
    sns.lineplot(x=aylar, y=ortalama_verimlilik, marker='o', label=f'Makine {MachCode}')
    
    plt.title(f'{MachCode} nolu Makine için Aylara Göre Ortalama Verimlilik')
    plt.xlabel('Ay')
    plt.ylabel('Aylık Ortalama Verimlilik')
    plt.legend(loc='best')
    plt.show()

for i in df["MachCode"].unique():
    makine_verimlilik(i)

Yüzdelik dilime göre CycleTime değerleri

In [ ]:
percentiles = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
cycletime_percentiles = df['CycleTime'].quantile([p / 100 for p in percentiles])
plt.figure(figsize=(10, 6))
plt.plot(percentiles, cycletime_percentiles, marker='o')
plt.title('CycleTime Yüzdelik Dilim Grafiği')
plt.xlabel('Yüzdelik Dilim')
plt.ylabel('CycleTime Değerleri')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# Ürün analizi

In [ ]:
df.CycleTime.min(), df.CycleTime.max()

CycleTime yani her bir eşsiz ürünün dönemsel olarak üretim sürelerini incelemek için sinüs eğrisi kullandım.
Çünkü sinüs eğrisi düzenli dönemsel davranışları temsil edebilir. 

In [ ]:
aylara_göre_cycletime_ort = df.groupby("Ay")["CycleTime"].mean().reset_index()
A, B, C = np.polyfit(aylara_göre_cycletime_ort["Ay"], aylara_göre_cycletime_ort["CycleTime"], 2) 
#2. dereceden bir polinom kullandık çünkü parabolik bir eğri görmek istiyoruz. 

sinus_katsayisi = 1 / (A * 2)
sinus_periyodu = 2 * np.pi / B


x = np.linspace(1, len(aylara_göre_cycletime_ort["CycleTime"]), 5000)
y = sinus_katsayisi * np.sin(B * x + C)

plt.figure(figsize=(12, 6))
sns.lineplot(x="Ay", y="CycleTime", data=aylara_göre_cycletime_ort, label="Üretim Süreleri")
sns.lineplot(x=x, y=y, label="Sinüs Yaklaşımı", palette="muted")

plt.xlabel("Zaman")
plt.ylabel("Üretim Süreleri")
plt.title("Üretim Süreleri ve Sinüs Yaklaşımı")
plt.legend()

plt.show()


Ürünlerin yapılış sürelerinin uzunluğuna göre zorluk scalalarının belirlenmesi

In [ ]:
productcode_cycletime_ort = df.groupby("ProductCode")["CycleTime"].mean().to_dict()
max_cycletime = df["CycleTime"].max()
min_cycletime = df["CycleTime"].min()
df["Zorluk_Seviyesi"] = df["ProductCode"].map(productcode_cycletime_ort).apply(lambda x: round(1 + 9 * (x - min_cycletime) / (max_cycletime - min_cycletime)))


Hangi ürün daha çok üretilmiş?

In [ ]:
ürün_üretim_sayısı = df['ProductCode'].value_counts().reset_index()
ürün_üretim_sayısı.columns = ['ProductCode', 'ÜretimSayısı']
ürün_üretim_sayısı.head(10)


İplik çeşidine göre Efficieny ve CycleTime değerleri nasıl? Hangi iplik çeşidi daha fazla verimlilik sağlamış?

In [ ]:
ort_efficiency_yarncode = df.groupby("YarnCode")["Efficiency"].mean().reset_index()
ort_efficiency_yarncode.sort_values(by="Efficiency", ascending=False)

Hangi iplik çeşidi daha uzun sürede üretilmiş?

In [ ]:
ort_CycleTime_yarncode = df.groupby("YarnCode")["CycleTime"].mean().reset_index()
ort_CycleTime_yarncode.sort_values(by="CycleTime", ascending=False)

Efficency ile CycleTime arasında bir ilişki var mı? 
 (  Yani en düşük cycle time için en verimli üretim diyebilir miyiz? )


In [ ]:
import scipy.stats as stats

korelasyon, p_deger = stats.pearsonr(df["Efficiency"], df["CycleTime"])
print(f"Pearson Korelasyon Katsayısı: {korelasyon}")

if p_deger < 0.05:
    print("Efficency ile Cycle arasında anlamlı bir korelasyon vardır.")
else:
    print("Efficency ile Cycle arasında anlamlı bir korelasyon yoktur.")

In [ ]:
correlation = df["CycleTime"].corr(df["Efficiency"])

abs_correlation = abs(correlation)

print("Korelasyon Katsayısı:", correlation)
print("Mutlak Korelasyon Katsayısı:", abs_correlation)

In [ ]:
subset = df[['CycleTime', 'Efficiency']]

sns.pairplot(subset, kind='scatter')
plt.show()

* Beklenildiği gibi CycleTime düştükçe Efficiency yükseldiği gözlemleniyor. Yani Ne kadar düşük Cycletime o kadar verim.
* 2023 Mart-Nisan aylarında Efficiency pik noktaya ulaşıyor ve sonrasında ekim 2023'e kadar yaklaşık %10'luk bir düşüş gözlemleniyor.
* YarnCode yani iplik çeşidinin doğrudan Efficiency'ye etki ettiğini gözlemleyebiliriz. 

# Feature Engineering işlemleri

https://en.wikipedia.org/wiki/Multivariate_normal_distribution #Multivariate normallik dağılımı

In [ ]:
 df = df.drop("Shift", axis=1, inplace=True)
 df=df.drop(["GroupCode","Item","adet"], axis=1,inplace=True)
 cat_list = ["GroupCode","ProductCode","YarnCode","Ability","Platin","Yıl","Ay","Haftanın_Günü","Vardiya_Saati"]
 for i in cat_list:
    cat_dummies = pd.get_dummies(df[i], prefix=var)
    df = pd.concat([df, cat_dummies], axis=1)
    df = df.drop(i, axis=1)

In [ ]:
## Tamamlanmadı

#from statsmodels.tsa.holtwinters import ExponentialSmoothing
#machines = df['Makine_Code'].unique()

#train_data = df[df['Date'] <= '2023-09-01']

#for machine in machines:
    #filtered_data = train_data[train_data["Makine_Code"] == machine]
    #model = ExponentialSmoothing(df["Efficiency"], trend="add", seasonal="add", seasonal_periods=25)
    #model_fit = model.fit(optimized=True, use_boxcox=True)
    #forecast = model_fit.forecast(step=25)
    #forecast_dates = pd.date_range(start='2023-09-01', periods=25, freq='D')
    #forecast_data = pd.DataFrame({'Date': forecast_dates, 'Efficiency_Forecast': forecast})
    #return forecast_data